In [158]:
from pytube import YouTube
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from IPython.display import clear_output
import os, glob
import speech_recognition as sr
import ffmpy
import wave
import contextlib
import random
from google.cloud import speech
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [4]:
links = []

In [5]:
driver = webdriver.Chrome('G:\My Drive\Desktop\Tools\chromedriver89.exe');
driver.get('https://www.youtube.com/c/TED/videos'); time.sleep(3);
#driver.get('https://www.youtube.com/c/TED/videos?view=0&sort=da&flow=grid'); time.sleep(3)

page = []

while len(page)<1:
    driver.find_element_by_css_selector("div#search-input > input#search").send_keys(Keys.PAGE_DOWN);
    page.append(driver.find_elements_by_xpath("/html//div[@id='details']/div[@id='meta']//a[@href]"));
    clear_output()
    print(f'Total pages scraped: {len(page)}');
for x in page[-1]:
    if x.get_attribute('href') not in links:
        links.append(x.get_attribute('href'))
    else:
        pass
driver.close()
driver.quit()

Total pages scraped: 1


In [7]:
pd.DataFrame(links).to_csv(f'TEDx_links.csv', index = False)
data = pd.read_csv(f'TEDx_links.csv')
data.head()

,0
0,https://www.youtube.com/watch?v=Qm02X0aE8uU
1,https://www.youtube.com/watch?v=Axh07mJ9Ag4
2,https://www.youtube.com/watch?v=Mkelhs_OVMc
3,https://www.youtube.com/watch?v=g4xGbbDACDw
4,https://www.youtube.com/watch?v=CpzjPnDW1G4


In [75]:
str(data.loc[0]).split(' ')[4].split('\n')[0] 
#This feels too long just to query the value at index as string instead of object.

'https://www.youtube.com/watch?v=Qm02X0aE8uU'

In [11]:
print(f'Total links: {len(data)}')

Total links: 30


In [166]:
print(files[0:5])

['Can we learn to talk to sperm whales  David Gruber.mp4', '4 myths and misunderstandings about doing business in Africa  Nomava Zanazo.mp4', 'The giant leaps in language technology -- and whos left behind  Kalika Bali.mp4']


In [171]:
#Download videos from YouTube using scraped link list
for x in range(5):
    yt = YouTube(str(data.loc[x]).split(' ')[4].split('\n')[0])
    yt.streams.filter(only_audio=True)[0].download('rawFiles')
    clear_output()
    print(f'Downloaded {x+1} videos.')

#Remove old files to make a clean batch
for file in [x for x in os.listdir('rawFiles') if x.split('.')[-1]!='mp4']:
    os.remove(f'rawFiles\\{file}')

#Convert .mp4 to .wav and encoding filename for processing
for idx, file in enumerate([x for x in os.listdir('rawFiles') if x.split('.')[-1]=='mp4']):
    ff = ffmpy.FFmpeg(executable="drivers\\ffmpeg.exe",\
                      inputs = {f"rawFiles\\{file}" : None},\
                      outputs = {f"rawFiles\\{idx}.wav" : f'-acodec pcm_s16le -ac 1 -ar 16000'})
    ff.run()
    os.remove(f'rawFiles\\{file}')
print('All videos converted.')

Downloaded 5 videos.
All videos converted.


In [384]:
#Method1: Splitting by a fixed time length.
cut_len = 10 #seconds

for idx, file in enumerate([x for x in os.listdir('rawFiles') if x.split('.')[-1]=='wav']):
    with contextlib.closing(wave.open(f'rawFiles\\{file}','r')) as f: #opening the file to read
        duration = round(f.getnframes() / float(f.getframerate())) #total frames / the framerate
    x=0
    while x<duration/cut_len: #doing it this way to have x as 1 or could have simply added cut_len to x at each iter
        ff = ffmpy.FFmpeg(executable="drivers\\ffmpeg.exe",\
                          inputs = {f"rawFiles\\{file}" : None},\
                          outputs = {f"splitFiles\\{idx}_{x}.wav" : f'-ss {12+x*cut_len} -t {cut_len}'}) # -ss(starting at) -t(seconds to keep) that 12 is to crop out TEDx intro sound :P
        ff.run()
        x+=1

In [379]:
#Method2: Splitting by silence in audio.

'''for file in [x for x in os.listdir('rawFiles') if x.split('.')[-1]=='wav']:
    song = AudioSegment.from_wav(f"rawFiles\\{file}")
    chunks = split_on_silence(song, min_silence_len = 800, silence_thresh = -50)
    for idx, chunk in enumerate(chunks):
        chunk.export(f"splitFiles\\{file}_{idx}.wav", format='wav')'''

In [157]:
files = [x for x in os.listdir('splitFiles')]

transcript = []
r = sr.Recognizer()

for file in files:
    temp=[]
    audio = sr.AudioFile(f'splitFiles\\{file}')
    with audio as source:
        audio_file = r.record(source)
        try:
            temp = (r.recognize_google(audio_file))
            transcript.append(temp)
        except:
            pass
print(len(transcript))
print(transcript[0:50])

207
['discount', 'aatankwad chief editor and family', 'school bus', 'most sexiest performance ever existed', 'is the distance of the my is palm oil from weather in our and this is amazing the complex matriarchal society', 'Scott Thomas working communication', 'new design', "please don't get me now what's time", 'factory and find out', 'next five years our team of Specialists provide a linguist Marine biologist intuition Technologies make contact with another species and hopefully', 'Mere Kata', 'people nature which can change your perspective about self and reshape', 'relationship', 'online punishment', 'Vishwa course in pattern in Bengal', 'Hip Hop recent make contact with other animals for hundreds of years', 'how can we do what others cannot', "especially given I'm sitting here on my couch in New York city", 'animal that have time I can purchase', 'I spent the last few years as a Marine biologist and oceanographer sitting Roshan from all the two perspectives from microbes short', 'i

In [373]:
transcript[random.randint(0, len(transcript))]

'founded the Uganda Museum which became the most important cultural and linguistic Centre for the language and its community'